In [1]:
#@title Run this to download data and prepare our environment!  { display-mode: "form" }

import cv2
import dlib
import pickle
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import itertools 

import urllib.request
 
from sklearn import metrics
from scipy.spatial import distance
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from tqdm import tqdm,tqdm_pandas
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import re
import keras

from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model

# grab tools from our tensorflow and keras toolboxes!
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import optimizers

warnings.filterwarnings("ignore")

'''
Plots the confusion Matrix and saves it
'''
def plot_confusion_matrix(y_true,y_predicted):
  cm = metrics.confusion_matrix(y_true, y_predicted)
  print ("Plotting the Confusion Matrix")
  labels = list(label_map.values())
  df_cm = pd.DataFrame(cm,index = labels,columns = labels)
  fig = plt.figure()
  res = sns.heatmap(df_cm, annot=True,cmap='Blues', fmt='g')
  plt.yticks([0.5,1.5,2.5,3.5,4.5], labels,va='center')
  plt.title('Confusion Matrix - TestData')
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
 
  plt.show()
  plt.close()

def plot_graphs(history, best):
  
  plt.figure(figsize=[10,4])
  # summarize history for accuracy
  plt.subplot(121)
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('model accuracy across training\n best accuracy of %.02f'%best[1])
  plt.ylabel('accuracy')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  
  # summarize history for loss
  plt.subplot(122)
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('model loss across training\n best loss of %.02f'%best[0])
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

#Integer to Label Mapping
label_map = {"0":"ANGRY","1":"HAPPY","2":"SAD","3":"SURPRISE","4":"NEUTRAL"}

In [2]:
# we'll use the same epochs and batch size as above
width, height = 160, 128

In [3]:
## X_train_cnn = X_train.reshape(len(X_train),height,width)
## X_test_cnn = X_test.reshape(len(X_test),height,width)

# we've converted them to images
## print(X_train_cnn.shape) 
# now we add one more dimension for model compatibility
## print(X_test_cnn.shape) 

train_ds = tf.keras.utils.image_dataset_from_directory(  
  "C:\\Users\\jaden\\OneDrive\\Documents\\GitHub\\ScienceFair_AI_Models_2023\\jpgTrainingData\\Train",  
  seed=123,  
  image_size = (height, width),
  batch_size = 2, color_mode = "grayscale")

val_ds = tf.keras.utils.image_dataset_from_directory(  
  "C:\\Users\\jaden\\OneDrive\\Documents\\GitHub\\ScienceFair_AI_Models_2023\\jpgTrainingData\\Val",  
  seed=123,  
  image_size = (height, width),
  batch_size = 1, color_mode = "grayscale") 

test_ds = tf.keras.utils.image_dataset_from_directory(  
  "C:\\Users\\jaden\\OneDrive\\Documents\\GitHub\\ScienceFair_AI_Models_2023\\jpgTrainingData\\Test",  
  seed=123,  
  image_size = (height, width),
  batch_size = 1, color_mode = "grayscale")  

Found 90 files belonging to 2 classes.
Found 10 files belonging to 2 classes.
Found 12 files belonging to 2 classes.


In [4]:
# now we add one more dimension for model compatibility
## X_train_cnn = np.expand_dims(X_train_cnn,3)
## X_test_cnn = np.expand_dims(X_test_cnn,3)

## print(X_train_cnn.shape)

In [4]:
#@title Instructor Solution

# pedagogically, it's probably a good idea to not use this model, but it's nice to have in case
# comparing the basic model above with transfer learning more clearly motivates that approach

cnn_model = Sequential()

cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(height, width, 1), kernel_regularizer=l2(0.01)))
cnn_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.5))

cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.5))

cnn_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.5))

cnn_model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
cnn_model.add(Dropout(0.5))

cnn_model.add(Flatten())

cnn_model.add(Dense(512, activation='relu'))
cnn_model.add(Dropout(0.4))
cnn_model.add(Dense(256, activation='relu'))
cnn_model.add(Dropout(0.4))
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(1, activation='softmax'))


In [5]:
#Saves the Best Model Based on Val Loss
checkpoint = ModelCheckpoint('best_cnn_model.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

# compliling the model with adam optimizer and categorical crossentropy loss
cnn_model.compile(loss=binary_crossentropy, optimizer=Adam(lr=0.01, beta_1=0.9, beta_2=0.999), metrics=['accuracy'])

# training the model
cnn_history = cnn_model.fit(train_ds, validation_data = val_ds, batch_size=2, epochs=50, verbose=1, 
                            callbacks=[checkpoint], shuffle=True)

Epoch 1/50
45/45 [==============================] - ETA: 0s - loss: 11.3483 - accuracy: 0.3222
Epoch 1: val_loss improved from inf to 709.17450, saving model to best_cnn_model.h5
45/45 [==============================] - 24s 463ms/step - loss: 11.3483 - accuracy: 0.3222 - val_loss: 709.1745 - val_accuracy: 0.3000
Epoch 2/50
45/45 [==============================] - ETA: 0s - loss: 12.1436 - accuracy: 0.3222
Epoch 2: val_loss improved from 709.17450 to 649.73535, saving model to best_cnn_model.h5
45/45 [==============================] - 21s 472ms/step - loss: 12.1436 - accuracy: 0.3222 - val_loss: 649.7354 - val_accuracy: 0.3000
Epoch 3/50
45/45 [==============================] - ETA: 0s - loss: 8.1994 - accuracy: 0.3222
Epoch 3: val_loss improved from 649.73535 to 28.60740, saving model to best_cnn_model.h5
45/45 [==============================] - 22s 490ms/step - loss: 8.1994 - accuracy: 0.3222 - val_loss: 28.6074 - val_accuracy: 0.3000
Epoch 4/50
45/45 [==============================] 

45/45 [==============================] - ETA: 0s - loss: 0.6353 - accuracy: 0.3222
Epoch 29: val_loss did not improve from 0.55147
45/45 [==============================] - 24s 544ms/step - loss: 0.6353 - accuracy: 0.3222 - val_loss: 0.6443 - val_accuracy: 0.3000
Epoch 30/50
45/45 [==============================] - ETA: 0s - loss: 0.9529 - accuracy: 0.3222
Epoch 30: val_loss did not improve from 0.55147
45/45 [==============================] - 24s 529ms/step - loss: 0.9529 - accuracy: 0.3222 - val_loss: 0.6979 - val_accuracy: 0.3000
Epoch 31/50
45/45 [==============================] - ETA: 0s - loss: 0.7234 - accuracy: 0.3222
Epoch 31: val_loss did not improve from 0.55147
45/45 [==============================] - 25s 550ms/step - loss: 0.7234 - accuracy: 0.3222 - val_loss: 0.7106 - val_accuracy: 0.3000
Epoch 32/50
45/45 [==============================] - ETA: 0s - loss: 0.7298 - accuracy: 0.3222
Epoch 32: val_loss did not improve from 0.55147
45/45 [==============================] - 26s

In [7]:
#@title Instructor Solution
cnn_performance = cnn_model.evaluate(X_test_cnn, y_test, batch_size=64)

NameError: name 'X_test_cnn' is not defined

In [ ]:
#@title Instructor Solution
plot_graphs(cnn_history, cnn_performance)

In [ ]:
#@title Run this to build your transfer learning model { display-mode: "form" }

import keras
from keras.models import Sequential
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape, Dense, Conv2D, GlobalAveragePooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.applications.vgg16 import VGG16

# load the vgg network that is an 'expert' at 'imagenet' but do not include the FC layers
vgg_expert = VGG16(weights = 'imagenet', include_top = False, input_shape = (48, 48, 3))

# we add the first 12 layers of vgg to our own model vgg_model
vgg_model = Sequential()
vgg_model.add(vgg_expert)

# and then add our own layers on top of it
vgg_model.add(GlobalAveragePooling2D())
vgg_model.add(Dense(1024, activation = 'relu'))
vgg_model.add(Dropout(0.3))
vgg_model.add(Dense(512, activation = 'relu'))
vgg_model.add(Dropout(0.3))
vgg_model.add(Dense(5, activation = 'sigmoid'))

# finally, we build the vgg model and turn it on so we can use it!
vgg_model.compile(loss = 'categorical_crossentropy', 
          optimizer = SGD(lr=1e-4, momentum=0.95), 
          metrics=['accuracy'])

X_TRAIN = np.array([np.transpose(np.array([X_train_cnn[ix].squeeze() for i in range(3)]), (1,2,0)) for ix in range(len(X_train))])
X_TEST = np.array([np.transpose(np.array([X_test_cnn[ix].squeeze() for i in range(3)]), (1,2,0)) for ix in range(len(X_test))])

#training the model
vgg_history = vgg_model.fit(X_TRAIN, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          callbacks=[checkpoint],
          validation_data=(X_TEST, y_test),
          shuffle=True)

In [ ]:
#@title Instructor Solution
vgg_performance = vgg_model.evaluate(X_TEST, y_test, batch_size=64)

In [ ]:
#@title Instructor Solution
plot_graphs(vgg_history, vgg_performance)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# pixel-based knn classification accuracy... this will take a few minutes 
knn_pixel = KNeighborsClassifier(n_neighbors=3)
knn_pixel.fit(X_train, y_train)
knn_predictions_pixel = knn_pixel.predict(X_test)
knn_accuracy_pixel = np.mean(knn_predictions_pixel.argmax(1) == y_test.argmax(1))

# landmark-based knn classification accuracy... this will take a few minutes 
knn_lm = KNeighborsClassifier(n_neighbors=3)
knn_lm.fit(X_train_lm, y_train_lm)
knn_predictions_lm = knn_lm.predict(X_test_lm)
knn_accuracy_lm = np.mean( knn_predictions_lm.argmax(1) == y_test_lm.argmax(1))

# chance accuracy
chance_accuracy = 1/n_labels

# human accuracy
human_accuracy = .65 

In [ ]:
def compare_learning(mlp, lm, cnn, vgg, benchmarks): 

  plt.figure(figsize=[7, 5])  

  # summarize history for accuracy

  plt.xticks(range(4, epochs+1,5), range(5, epochs+1,5) )
  
  plt.title('Performance of multiple models across training', y=1.05, fontsize=15)

  plt.ylim([.15, .72])

  plt.plot(lm.history['val_accuracy'], linestyle='-', color='purple', )
  plt.plot(mlp.history['val_accuracy'], linestyle='--', color='purple')

  plt.plot(vgg.history['val_accuracy'],color='blue', linestyle='-', linewidth=2)
  plt.plot(cnn.history['val_accuracy'],color='red', linestyle='-', linewidth=2)
  
  plt.ylabel('Accuracy', fontsize=15)
  plt.xlabel('Training Epoch', fontsize=15)


  plt.legend(['MLP''$_{landmarks}$', 'MLP''$_{pixels}$', 'CNN''$_{pretrained}$', 'CNN''$_{from.scratch}$'],
             framealpha=0, title_fontsize=9,loc=8)

  plt.xticks(range(4, epochs+1,5), range(5, epochs+1,5) )
  plt.ylim([.11, .72])

  for i_benchmark in benchmarks: 
    plt.scatter(x=epochs, y=references[i_benchmark], color='black', marker='_', s=200) 
    plt.annotate('%s: %d%%'%( i_benchmark, references[i_benchmark] *100), xy=(epochs+1, references[i_benchmark]-.005), size=10, color='black', ha='left') 
  plt.annotate('Important References', xy=(epochs+.5, .69), size=10, color='black', ha='left', fontweight='bold') 

  plt.ylim([.11, .72])
  plt.gca().spines['right'].set_color('none')
  plt.gca().spines['top'].set_color('none')

  plt.show()

In [ ]:
references = {'$KNN_{pixel}$ Accuracy': knn_accuracy_pixel, '$KNN_{landmark}$ Accuracy': knn_accuracy_lm, 'Human Accuracy': human_accuracy, 'chance': chance_accuracy}

In [ ]:
compare_learning(mlp_history, lm_history, cnn_history, vgg_history, references)